### 앙상블
* 보팅, 배깅, 부스팅으로 구분 + 스태깅도 있음
* 대표적인 배깅은 랜덤 포레스트 알고리즘이 있음. 부스팅은 에이다 부스팅, 그래디언트 부스팅, XGboost, LightGBM 등\
(정형 데이터의 분류나 회귀에서는 GBM 부스팅 계열의 앙상블이 전반적으로 높은 예측 성능)\
(배깅은 병렬, 부스팅은 순차적 학습)\
(부스팅은 오답에 대해 높은 가중치를 부여하고, 정답에 낮은 가중치를 부여함\
오답을 정답으로 맞추기 위해 오답에 더 집중하게 됨\
부스팅은 배깅에 비해 에러가 적고 성능이 좋지만, 속도가 느리고 오버피팅 가능성이 있음)

* 단일 모델의 약점으르 다수의 모델들을 결합하여 보완
* 뛰어난 성능을 가진 모델들로만 구성하는 것보다 성능이 떨어지더라도 서로 다른 유형의 모델을 섞는 것이 전체 성능에 도움 될 수 있음

#### 보팅 sklearn.ensemble.VotingClassifier
* class sklearn.ensemble.VotingClassifier(estimators, *, voting='hard', weights=None, n_jobs=None, flatten_transform=True, verbose=False)

In [13]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [14]:
import warnings
warnings.filterwarnings('ignore')

#개별 모델은 로지스틱 회귀와 KNN
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=4)

#개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR',lr_clf),('KNN',knn_clf)],voting='soft')
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=11)

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print(f'voting 분류기 정확도 : {accuracy_score(y_test,pred):.4f}')

classifiers = [lr_clf,knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print(f'{class_name} 정확도 : {accuracy_score(y_test,pred):.4f}')

voting 분류기 정확도 : 0.9561
LogisticRegression 정확도 : 0.9474
KNeighborsClassifier 정확도 : 0.9298


#### 배깅 sklearn.ensemble.BaggingClassifier
* class sklearn.ensemble.BaggingClassifier(base_estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)
* 대표적 알고리즘 : 랜덤 포레스트 ( 다재 다능 알고리즘, 앙상블 알고리즘 중 비교적 빠른 수행 속도, 다양한 영역에서 높은 예측 성능)

#### Random Forest sklearn.ensemble.RandomForestClassifier
* class sklearn.ensemble.RandomForestClassifier(n_estimators=100(결정트리 개수 지정), *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto'(기본값이 sqrt(전체피처개수)), max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
* 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정
* 개별 분류기의 기반 알고리즘은 결정트리 but 개별 트리가 학습하는 데이터 셋은 전체 데이터에서 일부가 중첩되게 샘플링된 데이터 셋\
- 여러개의 데이터 셋을 중첩되게 분리하는 것을 부트스트래핑 분할 방식
* 원본 데이터의 건수가 10개인 학습 데이터 셋에 랜덤 포레스트를 3개의 결정 트리 기반으로 학습하려고  n_estimators = 3 으로 하이퍼 파라미터 부여할 수 ㅇ

In [15]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)

    #drop 할 때 axis 넣어줘야해
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

In [16]:
import pandas as pd
def get_human_dataset():

    #데이터 파일 공백으로 분리되어 있음
    feature_name_df = pd.read_csv('datasets/human_activity/features.txt',sep='\s+', header=None, names=['column_index','column_name'])

    #중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    #데이터 프레임에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()

    #학습 피처 데이터 셋과 테스트 피처 데이터를 DataFrame으로 로딩, 컬럼명은 feature_name 적용
    X_train = pd.read_csv('datasets/human_activity/train/X_train.txt',sep='\s+', names=feature_name)
    X_test = pd.read_csv('datasets/human_activity/test/X_test.txt',sep='\s+', names=feature_name)

    #학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 컬럼명은 action으로 부여 
    y_train = pd.read_csv('datasets/human_activity/train/y_train.txt',sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('datasets/human_activity/test/y_test.txt',sep='\s+', header=None, names=['action'])

    #로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'랜덤 포레스트 정확도 : {accuracy :.4f}')

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}
#randomclassifier 객체 생성 후 gridsearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print(f'최적 하이퍼 파라미터 : \n {grid_cv.best_params_}')
print(f'최고 예측 정확도 : {grid_cv.best_score_:.4f}')

최적 하이퍼 파라미터 : 
 {'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도 : 0.9270


In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, min_samples_split=8, random_state=0)
rf_clf1.fit(X_train,y_train)
pred = rf_clf1.predict(X_test)
print(f'예측 정확도 : {accuracy_score(y_test, pred):.4f}')

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

ftr_importances_value = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_value,index=X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

NameError: name 'rf_clf1' is not defined

In [21]:
ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(7,7))
plt.title('Feature importances top 20')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

NameError: name 'ftr_importances' is not defined

In [22]:
ftr_top20.index

NameError: name 'ftr_top20' is not defined

In [23]:
X_df=pd.read_csv('datasets/human_activity/train/X_train.txt',sep='\s+')
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7351 entries, 0 to 7350
Columns: 561 entries, 2.8858451e-001 to -5.8626924e-002
dtypes: float64(561)
memory usage: 31.5 MB
